# Processing specs

This notebook serves to establish named sets of processing parameters. The idea is that, for example, `onescintle` refers to a specific amount of averaging and other processing settings; this way multiple sets of settings can coexist and be documented. There are actually two levels: the `processing_name`, which defines the alignment, zapping, and averaging, and the `toa_name`, which defines the template and TOA generation method. Each allows different settings to be used for each telescope and band.

In [1]:
%load_ext autoreload

%autoreload 2

import os
from glob import glob
import subprocess
import shutil
import traceback
import random
import pickle
from os.path import join

import joblib
from concurrent.futures import ProcessPoolExecutor

from backports import tempfile

import matplotlib
matplotlib.rcParams['savefig.dpi'] = 144
matplotlib.rcParams["image.composite_image"]=False
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize
from numpy.fft import rfft, irfft, fft, ifft

import psrchive
import residuals

import pipe
from pipe import rotate_phase, convert_template, align_profile, align_scale_profile

plt.viridis()

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


In [2]:
processing_specs = {}

In [3]:
wideband = dict(generic=dict(cal_db="data/cal/cal-noflux.db"))
wideband["AO",1400] = {}
wideband["GBT",1400] = {}
wideband["WSRT",1400] = dict(toa_time=600.)

processing_specs["wideband"] = wideband


In [4]:
singlefrequency = dict(generic=dict(rm=30))
singlefrequency["AO",1400] = {}
singlefrequency["GBT",1400] = {}
singlefrequency["WSRT",1400] = dict(toa_time=600.)

processing_specs["singlefrequency"] = singlefrequency

In [5]:
subband = dict(generic=dict(toa_bw=100.,toa_time=60.,rm=30))
subband["AO",1400] = {}
subband["GBT",1400] = {}
subband["WSRT",1400] = dict(toa_bw=60., toa_time=1200.)
subband["AO",430] = dict(toa_bw=10.)
subband["WSRT",350] = dict(toa_bw=30., toa_time=1200.)

processing_specs["subband"] = subband


In [6]:
# AO has plenty of SNR
# WSRT is okay for long observations but short ones can't really be used
# GBT loses many subints to SNR<10
onescintle = dict(generic=dict(toa_bw=5.,toa_time=1200.,rm=30))
onescintle["AO",1400] = {}
onescintle["GBT",1400] = {}
onescintle["WSRT",1400] = dict(toa_bw=5., toa_time=7200.)
onescintle["AO",430] = dict(toa_bw=10.)
onescintle["WSRT",350] = dict(toa_bw=30., toa_time=1200.)

processing_specs["onescintle"] = onescintle

In [7]:
# Like onescintle, but allow wider subbands for a factor 2 in SNR
tuned = dict(generic=dict(toa_bw=20.,toa_time=1200.,rm=30))
tuned["AO",1400] = dict()
tuned["GBT",1400] = dict()
tuned["WSRT",1400] = dict(toa_time=7200.)
tuned["AO",430] = dict(toa_bw=np.inf)
tuned["WSRT",350] = dict(toa_bw=np.inf, toa_time=1200.)

processing_specs["tuned"] = tuned

In [27]:
sband = dict(generic=dict(toa_bw=40, rm=30, toa_time=1200))
sband["AO",2000] = dict()
sband["AO",3000] = dict(median_r=None)

processing_specs["sband"] = sband

In [8]:
with open("processing_specs.pickle","wb") as f:
    pickle.dump(processing_specs, f)

In [6]:
toa_specs = {}

In [7]:
normal = dict(generic={})
normal["AO",1400] = dict(template="template-work/56736.85_AO_1400.profile.psrsmooth")
normal["GBT",1400] = dict(template="template-work/56736.85_AO_1400.profile.psrsmooth")
normal["WSRT",1400] = dict(template="template-work/56736.85_AO_1400.profile.psrsmooth")
normal["AO",430] = dict(template="template-work/56070.64_AO_430.tscrunch.fscrunch.sm")
normal["AO",350] = dict(template="template-work/56070.64_AO_430.tscrunch.fscrunch.sm")
normal["WSRT",350] = dict(template="template-work/56070.64_AO_430.tscrunch.fscrunch.sm")

toa_specs["normal"] = normal

In [8]:
mueller = dict(generic=dict(match="mueller"))
mueller["AO",1400] = dict(template="template-work/56412.76_GBT_1400.rmset.scrunch.sm")
mueller["GBT",1400] = dict(template="template-work/56412.76_GBT_1400.rmset.scrunch.sm")
mueller["WSRT",1400] = dict(template="template-work/56412.76_GBT_1400.rmset.scrunch.sm")
mueller["AO",430] = dict(template="template-work/56070.64_AO_430.tscrunch.fscrunch.sm")
mueller["AO",350] = dict(template="template-work/56070.64_AO_430.tscrunch.fscrunch.sm")
mueller["WSRT",350] = dict(template="template-work/56070.64_AO_430.tscrunch.fscrunch.sm")

toa_specs["mueller"] = mueller

In [9]:
with open("toa_specs.pickle","wb") as f:
    pickle.dump(toa_specs, f)